**Artigo**: https://medium.com/@tarekeesa7/crewai-sharedmemory-groq-api-powerful-ai-agent-949ff19027c5

### **CrewAi + SharedMemory + Groq API = Powerful AI Agent**
____________________________________________________________

**Alunos**:

Gustavo Toledo Akiti   RA: 11201922303

Lucas da Silva Abdalla  RA: 11201920265

Paulo Victor Dias Soares  RA: 11202111146

### **Introdução**

O artigo explora a combinação de três tecnologias avançadas: CrewAi SharedMemory, e Groq API, para criar agentes de inteligência artificial (IA) altamente eficientes. O autor discute como a integração dessas ferramentas permite a criação de agentes que são capazes de realizar tarefas complexas com maior velocidade e precisão, destacando os benefícios específicos de cada tecnologia. A abordagem proposta visa melhorar o desempenho e a escalabilidade de aplicações de IA, tornando-as mais eficazes em diversos contextos.


### **Mudanças no código**



Ao executarmos o código inicialmente, notamos que algumas coisas não funcionavam tão bem ou poderiam ser melhoradas.

Por exemplo, no artigo, havia uma variável chamada "chat", que era mencionada no momento em que criamos os agentes. Essa variável não existia no código, mas vimos que ela representava o LLM que foi usado no programa. Por conta disso, ela foi substituida pela variável "llm" (mencionada no início de nosso código).

Além disso, criamos uma variável para a API Key do Groq, que anteriormente estava exposta no código, além de adicionar um "expected_output" na hora de definirmos a tarefa (email_task), já que, sem ela, não havia nenhum output criado.

Após a apresentação de nosso seminário, criamos uma versão modificada deste arquivo, que utiliza o Chat-GPT 3.5 Turbo. Naquela versão, há apenas 1 variação de e-mail criada. A versão modificada pode ser encontrada no nosso repositório do Github.

Por último, modificamos as instruções da tarefa "email_task", de forma que ela ficasse mais clara (o que parece ter funcionado, levando em conta os últimos testes).

### **Instalando os pactos iniciais**

O CrewAi é uma plataforma projetada para facilitar a colaboração entre múltiplos agentes de inteligência artificial (IA). Ele atua como um sistema de gerenciamento que coordena as tarefas distribuídas entre diferentes agentes, permitindo que eles trabalhem de forma conjunta para alcançar objetivos complexos.

A ideia é que, ao dividir tarefas entre agentes especializados e gerenciá-las de maneira eficiente, o desempenho geral do sistema de IA seja maximizado, tornando-o mais adaptável e eficiente em uma variedade de aplicações.

In [1]:
pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 663.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 34.8 MB/s eta 0:00:00

Instalaremos o DuckDuckGo, um tipo de motor de pesquisa, para podermos usar o "DuckDuckGoSearchRun".

O "DuckDuckGoSearchRun" vai ser utilizado como uma ferramenta de busca para ser usada pelos agentes.



In [2]:
pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 14.9 MB/s eta 0:00:00


O Groq será configurado para realizar tarefas de processamento de linguagem natural. Uma das vantagens desta API é que ela mais rápida do que a maioria das outras APIs.

In [3]:
pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 3.6 MB/s eta 0:00:00


### **Criando o projeto**

Importamos tudo o que será utilizado ao longo do projeto.

In [1]:
from crewai import Agent, Task, Crew, Process
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain.agents import  Tool
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.prompts import PromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


O ChatGroq vai ser utilizado para tarefas geração de texto. a temperatura = 0 vai fazer com que o modelo gere respostas mais determinísticas e previsíveis.

Aviso: Tentamos adicionar o "secret" chamado "GROQ_API_KEY", ao invés de colocar a API Key diretamente no código, mas isso resultou nos agentes de IA não conseguirem executar as suas funções (provavelmente

In [9]:
key = userdata.get('GROQ_API_KEY')
llm = ChatGroq(temperature=0, groq_api_key=key, model_name="mixtral-8x7b-32768")

Essa parte do código define como o histórico de uma conversa é armazenado e acessado para que o agente de IA possa gerar um resumo da conversa usando o Groq. O “ConversationBufferMemory()”  armazena o histórico de bate-papo do modelo e inicialize o “ReadOnlyMemory” para os agentes e suas ferramentas. O "ReadOnlyMemory" é um tipo de memória que permite acesso de leitura (não permite modificações) ao histórico da conversa armazenado no "ConversationBufferMemory".

In [10]:
template = """This is a conversation between a human and ai agent:
{chat_history}
Write a summary of the conversation for {input}:
"""
prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)
summary_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,
)

Agora, definimos e configuramos as ferramentas  que os agentes vão utilizar para executar as tarefas. Primeiro, criamos uma instância de pesquisa usando o DuckDuckGoSearchRun, que será usada para responder perguntas.

Em seguida, duas listas de ferramentas (tool_use e tool_use_1) são criadas. A primeira lista (tool_use) contém duas ferramentas: uma para realizar buscas na web (Search) e outra para resumir conversas (Summary).

A segunda lista (tool_use_1) contém apenas a ferramenta de resumo.

In [11]:
#create searches
search =  DuckDuckGoSearchRun()

tool_use = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Summary",
        func=summary_chain.run,
        description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.",
    )

]
tool_use_1 = [
    Tool(
        name="Summary",
        func=summary_chain.run,
        description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.",
    )
]

Definimos três agentes, cada um com um papel e objetivos específicos relacionados à criação de emails de marketing.

"O email_author" é configurado como um autor profissional de emails, responsável por criar emails concisos e envolventes, sem a capacidade de delegar tarefas, mas com acesso às ferramentas de busca e resumo.

O "marketing_strategist" atua como estrategista de marketing, liderando a equipe na criação de emails eficazes, com a capacidade de delegar tarefas e acesso apenas à ferramenta de resumo.

O "content_specialist" é responsável por criticar e refinar o conteúdo dos emails, também sem a capacidade de delegar, mas com acesso à ferramenta de resumo. Cada agente é equipado com um modelo de linguagem (Groq) e ferramentas específicas (definidas anteriormente) para realizar suas funções.

In [12]:

email_author = Agent(
    role='Professional Email Author',
    goal='Craft concise and engaging emails',
    backstory='Experienced in writing impactful marketing emails.',
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tool_use
)
marketing_strategist = Agent(
    role='Marketing Strategist',
    goal='Lead the team in creating effective cold emails',
    backstory='A seasoned Chief Marketing Officer with a keen eye for standout marketing content.',
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools=tool_use_1
)

content_specialist = Agent(
    role='Content Specialist',
    goal='Critique and refine email content',
    backstory='A professional copywriter with a wealth of experience in persuasive writing.',
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tool_use_1
)

Esta é a tarefa que os agentes vão realizar. Eles devem gerar 2 e-mails diferentes promovendo soluções para edição de vídeo. Após isso, o e-mail é avaliado e, baseado nas avaliações, 2 versões finais do e-mail são criadas (com as devidas correções).

O "marketing_strategist" vai receber esta tarefa e vai coemçar a delegar as funções de cada um dos agentes. O "email_author" vai escrever o e-mail e o "content_specialist" vai revisar o que foi escrito e adicionar os feedbacks necessários.

Esta tarefa pode ser modificada. As modificações resultarão em e-mails diferentes sendo criados (que atendem aos critérios especificados).

In [15]:
email_task = Task(
    description='''enerate two distinct variations of a cold email promoting a video editing solution, then evaluate the written emails for their effectiveness and engagement. Adjust the emails to align with best practices for cold outreach. Consider the feedback
    provided to the marketing_strategist. Revise the emails based on all feedback, creating two final versions.''',
    agent=marketing_strategist,  # O estrategista de marketing será o responsável por administrar esta tarefa
    expected_output="Two final versions of the cold email that meet the specified criteria."
)

### **Resultado**

Criamos uma equipe (Crew) composta pelos três agentes previamente definidos (email_author, marketing_strategist, content_specialist) e a atribuimos para realizar uma tarefa específica (email_task).

A equipe é configurada para operar de forma sequencial, onde cada agente executa sua função em uma ordem predeterminada. O resultado desse processo é armazenado na variável emails_output, e o programa imprime uma mensagem indicando que a equipe está trabalhando na tarefa de email.

**Importante**: As vezes, o output do programa mostra apenas a versão final do e-mail (revisada pelo "content_specialist"), e acaba não mostrando todo o processo de construção do e-mail.

In [16]:
email_crew = Crew(
    agents=[email_author, marketing_strategist, content_specialist],
    tasks=[email_task],
    verbose=True,
    process=Process.sequential
)

print("Crew: Working on Email Task")
emails_output = email_crew.kickoff()

Crew: Working on Email Task
 [2024-08-27 20:14:02][DEBUG]: == Working Agent: Marketing Strategist
 [2024-08-27 20:14:02][INFO]: == Starting Task: enerate two distinct variations of a cold email promoting a video editing solution, then evaluate the written emails for their effectiveness and engagement. Adjust the emails to align with best practices for cold outreach. Consider the feedback
    provided to the marketing_strategist. Revise the emails based on all feedback, creating two final versions.


> Entering new CrewAgentExecutor chain...
To generate effective cold emails, I need to consider the following best practices:

1. Personalize the email: Address the recipient by their first name and mention something specific about their company or role.
2. Keep it short and concise: Make sure the email is easy to read and understand, focusing on the main benefits of the video editing solution.
3. Provide value: Explain how the video editing solution can help the recipient's business or rol